In [1]:
import cv2 as cv
import os
import numpy as np

In [2]:
def preprocess_image_level(img_path):
    img = cv.imread(img_path)

    resized = cv.resize(img, (640, 640))

    contrast = cv.convertScaleAbs(resized, alpha=0.5, beta=50)

    grey = cv.cvtColor(contrast, cv.COLOR_BGR2GRAY)

    noise_reduction = cv.fastNlMeansDenoising(grey, None, 5, 7, 15)
    adaptive_thresh = cv.adaptiveThreshold(
        noise_reduction,
        255,
        cv.CALIB_CB_ADAPTIVE_THRESH,
        cv.THRESH_BINARY_INV,
        7,
        3,
    )

    nlabels, labels, stats, _ = cv.connectedComponentsWithStats(
        adaptive_thresh, None, None, None, 8, cv.CV_32S
    )

    sizes = stats[1:, -1]  # get CC_STAT_AREA component
    img2 = np.zeros((labels.shape), np.uint8)

    for i in range(0, nlabels - 1):
        if sizes[i] >= 40:  # filter small dotted regions
            img2[labels == i + 1] = 255

    dilate = cv.dilate(img2, (8, 8), iterations=3)

    erosion = cv.erode(dilate, (2, 2), iterations=1)

    result = cv.bitwise_not(erosion)

    return result

In [3]:
img_path = "/home/uwu/Desktop/object_detection/preview/yolo/1f9e8765-44db-4303-b12a-d6b30c9cc5f9.png"


In [4]:
mask = preprocess_image_level(img_path)

cv.imshow("test", mask)
cv.waitKey(0)
cv.destroyAllWindows()


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
